# CVPR2020比赛：商超检测基线
本项目基于深度学习平台飞桨（[PaddlePaddle](https://www.paddlepaddle.org.cn/)），利用一键式目标检测开发套件[PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)进行开发，欢迎使用并**star**。

## 基线方案介绍

本基线用于检测密集场景中产品的位置。基于飞桨推出的端到端目标检测开发套件[PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)进行二次开发，PaddleDetection提供了丰富的检测模型，包含目标检测、实例分割、人脸检测等100+个预训练模型，涵盖多种数据集竞赛冠军方案，同时还具备了高灵活度，通过模块化的设计解藕各个组建，基于配置文件即可轻松搭建各种检测模型。本基线对商超检测数据集进行读取和评估，采用faster_rcnn算法，ResNet50_vd作为主干网络，引入FPN模块，在测试集上检测的AP@.50:.05:.95达到0.382，AP@0.5达到0.599，AP@0.75达到0.440。可以选择PaddleDetection中提供的大量主干网络优化精度，也可以尝试PaddleDetection已具备的扩展特性进一步提升模型效果，例如group_norm, Modulated Deformable Convolution等，更多模型可以参考[MODEL_ZOO文档](https://github.com/PaddlePaddle/PaddleDetection/blob/release/0.4/docs/MODEL_ZOO_cn.md)。


| 基线方案 | 算法 | AP@.50:.05:.95 | AP@0.5 | AP@0.75 | 备注|
| ------- | ---- | -------- | -------- | -------- | -------- |
| 检测    | faster_rcnn + ResNet50_vd + FPN  | 0.332 | 0.574 |  0.352  |  基于[PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)进行二次开发  |



## 环境数据集和配置

AIStudio上已安装PaddlePaddle 1.8.4，右下角可看到版本号和python环境

### 1.配置路径

In [ ]:
%cd PaddleDetection

/home/aistudio/PaddleDetection


### 2.解压数据集

In [ ]:
import os
if os.path.exists("/home/aistudio/PaddleDetection/dataset/coco/images"):
    print("无需解压")
else:
    if os.path.exists("/home/aistudio/PaddleDetection/dataset/coco/annotations"):
         print("无需解压")
    else: 
        !unzip /home/aistudio/data/data54250/coco.zip -d /home/aistudio/PaddleDetection/dataset/
        print("解压完成")

数据集格式为COCO：

```txt

├── PaddleDetection/dataset/coco/
│   ├── images
│   └── annotations
│           ├── instances_train2017.json
│           └── instances_val2017.json
 ```
 
  - **注意：** 数据集已经为大家准备好，大家可以直接使用。

### 3.安装环境

In [ ]:
!pip install -r requirements.txt
!pip install pycocotools
!pip install Cython

Looking in indexes: https://mirror.baidu.com/pypi/simple/
Looking in indexes: https://mirror.baidu.com/pypi/simple/
Looking in indexes: https://mirror.baidu.com/pypi/simple/


## 检测方案流程

### 1.开始训练
我们使用了COCO数据集上训练好的模型做迁移学习，基线提供的配置需要在V100单卡上训练5小时左右，更多训练参数请参考[检测库入门使用文档](https://github.com/PaddlePaddle/PaddleDetection/blob/release/0.4/docs/tutorials/GETTING_STARTED_cn.md)：

**注意：**--eval为边训练边评估。

在训练中交替执行评估, 评估在每个snapshot_iter时开始。每次评估后还会评出最佳mAP模型保存到best_model文件夹下。

如果验证集很大，测试将会比较耗时，建议减少评估次数，或训练完再进行评估。

In [5]:
!python tools/train.py -c configs/obj365/cascade_rcnn_dcnv2_se154_vd_fpn_gn_cas.yml \
    -o pretrain_weights=output/cascade_rcnn_dcnv2_se154_vd_fpn_gn_cas/best_model --eval

2020-09-27 16:20:34,732-INFO: If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!


### 2.模型评估

In [ ]:
# 此处model_final可按需更改模型文件
!python tools/eval.py -c configs/yolov3_r34.yml -o weights=output/yolov3_r34/model_final

3、导出模型并打包

In [ ]:
!python tools/export_model.py -c configs/dcn/cascade_rcnn_dcn_r101_vd_fpn_1x.yml \
    -o weights=output/cascade_rcnn_dcn_r101_vd_fpn_1x/best_model

2020-09-26 16:32:19,478-INFO: save_inference_model pruned unused feed variables im_id
2020-09-26 16:32:19,479-INFO: Export inference model to output/cascade_rcnn_dcn_r101_vd_fpn_1x, input: ['image', 'im_info', 'im_shape'], output: ['multiclass_nms_0.tmp_0']...
2020-09-26 16:32:21,312-INFO: Not found annotation file annotations/instances_val2017.json, load coco17 categories.
2020-09-26 16:32:21,316-INFO: Export inference config file to output/cascade_rcnn_dcn_r101_vd_fpn_1x/infer_cfg.yml


In [ ]:
%cd output/cascade_rcnn_dcn_r101_vd_fpn_1x/
!tar -cvf output.tar infer_cfg.yml __model__ __params__

[Errno 2] No such file or directory: 'output/cascade_rcnn_dcn_r101_vd_fpn_1x/'
/home/aistudio/PaddleDetection/output/cascade_rcnn_dcn_r101_vd_fpn_1x
infer_cfg.yml
__model__
__params__
